In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
gwic_filename = 'data/missoula_valley_monitored_wells_data.csv'
usgs_filename = 'data/clark_fk_above_missoula_q.csv'
gw_df = pd.read_csv(gwic_filename, index_col='time', parse_dates=True)
q_df = pd.read_csv(usgs_filename, index_col='time', parse_dates=True)

In [3]:
df = pd.concat([q_df, gw_df], axis=1, join='inner')

In [8]:
df

,Q,151081,151179,151188,151204,67037,150967,151143,151203,157209,...,151190,151191,156566,157208,157212,157207,157211,157214,209254,209255
time,,,,,,,,,,,,,,,,,,,,,
1995-06-30,8235.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995-07-31,3910.0,6.97,21.53,5.08,NaN,20.08,NaN,19.32,12.95,NaN,...,64.09,28.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995-08-31,1530.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995-09-30,1535.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995-10-31,1610.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-30,6595.0,NaN,NaN,NaN,13.45,NaN,22.89,NaN,NaN,NaN,...,58.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07-31,2250.0,NaN,NaN,NaN,12.85,NaN,NaN,NaN,NaN,NaN,...,61.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-08-31,1290.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,64.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Data Imputation

## Regression imputation using Q and DOY 
Inspired by [Medium blog post](https://medium.com/@aaabulkhair/data-imputation-demystified-time-series-data-69bc9c798cb7)

In [4]:
reg_imputed = df.copy()

# Loop through columns
for c in np.arange(1, df.shape[1]):

    # Drop preceding NaNs
    first_ind = df.iloc[:, c].first_valid_index()
    new_df = df.iloc[:, [0, c]][first_ind:]  
        
    # Get indices of NaNs
    imputed_indices = new_df.iloc[:, 1][new_df.iloc[:, 1].isnull()].index

    # Drop rows with missing values
    df_dropped = new_df.iloc[:, [0, 1]].dropna()    

    # Get predictors (Q and doy) and dependent variable
    X = np.column_stack((df_dropped.Q.values, df_dropped.index.dayofyear.values))
    Y = df_dropped.iloc[:, 1].values

    # Instantiate model
    model = LinearRegression()

    # Fit model
    model.fit(X, Y)

    # Predict missing values
    pred1 = new_df.loc[imputed_indices, 'Q'].values
    pred2 = new_df.loc[imputed_indices, 'Q'].index.dayofyear.values
    preds = np.column_stack((pred1, pred2))
    predicted = model.predict(preds)

    # Fill missing values with predicted values
    reg_imputed.iloc[:, c][imputed_indices] = predicted

In [6]:
reg_imputed.to_csv('data/missoula_valley_wells_cleaned.csv')